## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
avail_lattice_test_number = np.copy(avail_lattice)
avail_lattice_test_number_flatten = avail_lattice_test_number.flatten()
avail_lattice_test_number_flatten [21] = 200
avail_lattice_test_number_reshaped = np.reshape(avail_lattice_test_number_flatten,(avail_lattice_test_number.shape) )
avail_lattice_test_number_reshaped

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 200],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  

## Env Lattices

In [3]:
Bv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Gv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Yv = np.random.randint(1, 99, (np.shape(avail_lattice)), dtype='l')
Rv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Vv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')

## Stencils

In [4]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [5]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [6]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [7]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

## Deriving all possible agent Center points

In [8]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
B1 = int(mass_size / 5)
G1= int( mass_size / 5)
Y1=int( mass_size / 7.5)
R1= int(mass_size / 7.5)
V1= int(mass_size -(B1+G1+Y1+R1))
Num_of_voxels_lst = np.array([B1,G1,Y1,R1,V1])
Cube_root_list = Num_of_voxels_lst ** (1/3)
Radius_list = Cube_root_list.astype(int) 
Maximum_radius = int(Radius_list[np.argmax(Radius_list)])

Num_of_voxels_lst

array([136, 136,  91,  91, 230])

In [9]:
#Divide the available lattice into grids to generate the necessary locations for origins

In [10]:
# All possible locations 
locations= [194,190,257,253,307,311]

## Agent Class

In [11]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = neighs_availibility_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(self.origin)
        
        

## Initilize Agents

In [12]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = origin
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [13]:
#Agent_2D = Agent_initilization 
Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,194 )
Agent_two=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,190 )
Agent_three=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,257)
Agent_four=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,253 )
Agent_five=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,307 )
Agent_six=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,311 )
# Agents at all 6 locs 

    

## Enviornment Class

In [14]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [15]:
all_permutations = (list(itertools.permutations([Agent_one,Agent_two,Agent_three,Agent_four,Agent_five,Agent_six], 5)))
#all_permutations

## Create Enviornments

In [16]:
# name the lattices myagent_attractor_one



# initiate the environment
# Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
#env_1 = environment(env_V, Agent_one,V1,"one_neighbour_occupy_cubish_behaviour")



## Run Simulations for all agents on all locations 

In [17]:
all_matrices= []
for agents in all_permutations :
    env_B = {"availibility": avail_lattice,"enviornment": Bv}
    env_G = {"availibility": avail_lattice,"enviornment": Gv}
    env_R = {"availibility": avail_lattice,"enviornment": Rv}
    env_Y = {"availibility": avail_lattice,"enviornment": Yv}
    env_V = {"availibility": avail_lattice,"enviornment": Vv}
    
    env_2 = environment(env_Y, agents[0],Y1,"one_neighbour_occupy_cubish_behaviour")
    env_3 = environment(env_G, agents[1],G1,"one_neighbour_occupy_cubish_behaviour")
    env_4 = environment(env_V, agents[2],V1,"one_neighbour_occupy_cubish_behaviour")
    env_5 = environment(env_B, agents[3],B1,"one_neighbour_occupy_cubish_behaviour")
    env_6 = environment(env_R, agents[4],R1,"one_neighbour_occupy_cubish_behaviour")
    env_availability_viz = []
    all_matrices.append(env_availability_viz)
    env_list =[env_2,env_3,env_4,env_5,env_6]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)

        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  

                env_availability_viz.append(e.availibility-1)

## Pickle Objects for reference 
change the name for each simulation

In [18]:
simulation = pickle.dump(all_matrices , open( "all_permutations.p", "wb" ) )

## Vizualise the simulation

In [19]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = sum(map(lambda e:e.number_of_iterations,env_list))
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

## Generate Cost matrix

In [20]:
cost_for_location_filter = [env_availability_viz[-1] == 310]
cost = np.sum(Bv[cost_for_location_filter])
cost

1

## Linear Optimization to derive a solution

In [21]:
from ortools.linear_solver import pywraplp


def main():
    # Data
    costs = [
        [624,631,612,649,638,604],
        [621,621,612,631,626,623],
        [379,393,392,391,384,382],
        [414,436,388,392,406,412],
        [1031,1057,1004,1051,1040,1021],
        [0,0,0,0,0,0]
    ]
    num_workers = 6

    num_tasks = len(costs[0])
    #print(num_tasks)
    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('Worker %d assigned to task %d.  Cost = %d' %
                          (i, j, costs[i][j]))


if __name__ == '__main__':
    main()
    


Total cost =  3140.0000000000005 

Worker 0 assigned to task 4.  Cost = 638
Worker 1 assigned to task 5.  Cost = 623
Worker 2 assigned to task 2.  Cost = 392
Worker 3 assigned to task 1.  Cost = 436
Worker 4 assigned to task 3.  Cost = 1051
Worker 5 assigned to task 0.  Cost = 0


## Total cost of permutation matrix

In [22]:
Final_frames=[]
for item in all_matrices:
    Final_frames.append(item[-1])
    

In [23]:
total_cost=[]
for item in Final_frames:
    cost_for_location_filter_1 = [env_availability_viz[-1] == 193]
    cost_for_location_filter_2 = [env_availability_viz[-1] == 189]
    cost_for_location_filter_3 = [env_availability_viz[-1] == 256]
    cost_for_location_filter_4 = [env_availability_viz[-1] == 252]
    cost_for_location_filter_5 = [env_availability_viz[-1] == 306]
    cost_for_location_filter_6 = [env_availability_viz[-1] == 310]

    cost = [(np.sum(Yv[cost_for_location_filter_1]))+(np.sum(Gv[cost_for_location_filter_2]))+(np.sum(Vv[cost_for_location_filter_3]))+(np.sum(Bv[cost_for_location_filter_4]))+(np.sum(Rv[cost_for_location_filter_5]))]
    total_cost.append(cost)

## Permutation Combinations for further simulation verifications

In [24]:
all_permutations = (list(itertools.permutations([194,190,257,253,307,311], 5)))
print(len(all_permutations))

720


In [25]:
Final_frames[0]

lattice([[[ -1, 189,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [26]:
Final_frames[2]

lattice([[[ -1, 256,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [27]:
all_matrices[719][-1]

lattice([[[ -1, 252,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [28]:
total_cost

[[8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 [8355],
 